In [1]:
import logging
import sys

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index import SimpleDirectoryReader

import oci
import ads
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
import oracledb

from config import INPUT_FILES
from config_private import COMPARTMENT_OCID, ENDPOINT

from oci_utils import load_oci_config

#this is the class realized for the integration with Llama-index
from oracle_vector_db import OracleVectorStore

In [2]:
# setup
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual
MODEL_NAME = "cohere.embed-english-v3.0"

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=MODEL_NAME,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={
        "service_endpoint": ENDPOINT
    },
)

In [3]:
pages = SimpleDirectoryReader(input_files=INPUT_FILES).load_data()

In [4]:
len(pages)

890

In [5]:
v_store = OracleVectorStore(verbose=False)

In [6]:
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={
        "service_endpoint": ENDPOINT
    },
)

In [7]:
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)
storage_context = StorageContext.from_defaults(vector_store=v_store)

In [8]:
index = VectorStoreIndex.from_documents(pages,
    storage_context=storage_context,
    service_context=service_context,
    show_progress=True
)

Parsing nodes:   0%|          | 0/890 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/894 [00:00<?, ?it/s]

In [ ]:
# before query, we need this
index.vector_store.persist()

In [9]:
query_engine = index.as_query_engine(similarity_top_k=5)

In [ ]:
%%time

question = "Write an email where you list new feature in Oracle Database 23c regarding JSON"

response = query_engine.query(question)

print(f"Question: {question}") 
print(response.response)
print("")